# Important Notice:
If the notebook fails to show the maps, please click on this link;
[Google Colaboratory](https://colab.research.google.com/drive/1Nbd4lPa-Q7o102xYbBnyzcmaHOovj94E?usp=sharing) in order to view the notebook.

# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
import folium
import wget
import pandas as pd
import numpy as np

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 8966 / 8966

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -100]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4.5)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=10, color='#0000FF', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)



Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
col = launch_sites_df.columns

# save coordinates in lists
sites = [v for k,v in launch_sites_df[col[0]].items()]
lat = [v for k,v in launch_sites_df[col[1]].items()]
long = [v for k,v in launch_sites_df[col[2]].items()]
circles = []

for i in range(len(sites)):
    coordinate = [lat[i], long[i]]
    # circle objects
    circle = folium.Circle(coordinate, radius=100, color='#000000', fill=True).add_child(folium.Popup(f'{sites[i]}'))
    
    # marker objects
    marker = folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), 
                                               html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f'{sites[i]}', ))
    site_map.add_child(circle)
    site_map.add_child(marker)
    
site_map

We can see that all launch sites are in very close proximity to the coast and they are also a couple thousand kilometres away from the equator line. It is interesting to see that most launch sites are concentrated near Miami.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    
    label = record[0]
    lat = record[1]
    long = record[2]
    row_color = record[4]
    
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location = [lat, long],
#         popup = label,
        icon=folium.Icon(color='white', icon_color=row_color)
    )
    marker_cluster.add_child(marker)

site_map.fit_bounds([[28.60577, -80.68102], [28.52813, -80.52168]]) # Insert resting coordinates for map
site_map

Clicking on the site, we can notice that **KSC LC-39A** had the highest success rate of rocket launches compared to other launch sites.

# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map.fit_bounds([[42.81152, -130.3418], [13.32548, -64.33594]])
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

coastline_distances = []

lat = [v for k,v in launch_sites_df[col[1]].items()]
long = [v for k,v in launch_sites_df[col[2]].items()]

coast_lat = [28.56225, 28.56334, 28.57339, 34.63284]
coast_long = [-80.5678, -80.56797, -80.60693, -120.62634]


for i in range(len(coast_long)):
    coastline_distances.append(calculate_distance(lat[i], long[i], coast_lat[i], coast_long[i]))
    

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 

for i in range(len(coast_long)):
    
    # nearest coastline coordinates
    coordinate = [coast_lat[i], coast_long[i]]
    
    # marker              
    distance_marker = folium.Marker(
       coordinate,
       icon=DivIcon(
           icon_size=(20,20),
           icon_anchor=(0,0),
           html='<div style="font-size: 12; color:#000080;"><b>%s</b></div>' % "{:10.2f} KM".format(coastline_distances[i]),
           )
       )
    
    site_map.add_child(distance_marker)
    

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate

for i in range(len(lat)):

    # coordinate
    coordinates = [[lat[i], long[i]], [coast_lat[i], coast_long[i]]]

    lines=folium.PolyLine(coordinates, weight=0.5)
    site_map.add_child(lines)

site_map.fit_bounds([[28.56798, -80.58229], [28.55791, -80.56224]])
site_map

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [ ]:
# enter coordinates
locations = []
locs_distances = []

railway_lat = [28.57211, 28.57211, 28.57307, 34.6338]
railway_long = [-80.58527, -80.58527, -80.65403, -120.62478]

highway_lat = [28.56278, 28.56278, 28.57359, 34.70472]
highway_long = [-80.57075, -80.57075, -80.6553, -120.56937]

city_lat = [28.39804, 28.39804, 28.60984, 34.63716]
city_long = [-80.60257, -80.60257, -80.80376, -120.45616]

for i in range(len(city_lat)):
    
    # save coordinates in a list object
    locs = [railway_lat[i], railway_long[i], '#BA55D3'], [highway_lat[i], highway_long[i], '#2E8B57'], [city_lat[i], city_long[i], '#DC143C']
    locations.append([locs])
    
    # save distances(km) in another list
    dist = [(calculate_distance(lat[i], long[i], railway_lat[i], railway_long[i])), 
    (calculate_distance(lat[i], long[i], highway_lat[i], highway_long[i])),
    (calculate_distance(lat[i], long[i], city_lat[i], city_long[i]))]
    
    locs_distances.append(dist)


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.

# loop through the location items for each launch site
for i in range(len(locations)):
    
    # site_items 
    site = locations[i]
    
    # site coordinates
    site_coordinates = [lat[i], long[i]]
    
    # loop through location item
    for i in range(len(site)):
    
        # nearest coordinates
        coordinates = site[i]
        
        # distance
        distances = locs_distances[i]
        
        # loop through each distance
        for i in range(len(distances)):
            
            # individual coordinate
            coordinate = coordinates[i][:2]
            
            # individual distance
            distance = distances[i]

#             print(coordinate)
#             print(distance)
#             break
            
            # marker              
            distance_marker = folium.Marker(
               coordinate,
               icon=DivIcon(
                   icon_size=(20,20),
                   icon_anchor=(0,0),
                   html=f'<div style="font-size: 5; color:{coordinates[i][2]};"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
                   )
               )

            
            # Draw a line between the marker to the launch site
            
            # coordinates from launch site to marker
            new_coordinates = [site_coordinates, coordinate]

            line = folium.PolyLine(new_coordinates, weight=0.5)

            site_map.add_child(line)
            site_map.add_child(distance_marker)
        

site_map.fit_bounds([[34.64242, -120.63443], [34.6229, -120.59301]])
site_map

 |  Map Object | Colour |
  | ----------------- | ------- |
 | Nearest Highway       | **<span style="color:#2E8B57">Green</span>**     |
  | Nearest Railway        | **<span style="color:#BA55D3">Purple</span>**    |
  | Nearest City        | **<span style="color:#DC143C"> Crimson </span>**     | 
  | Nearest Coastline        | **<span style="color:#000080">Dark Blue</span>**     | 




#### Observations:

* It appears that launch sites are usually set up atleast 18 km away from cities. This may be because of the desire to prevent any crashes near populated areas.
* It is also apparent that launch sites are in very close proximity to railways and highways. Perhaps, due to the necessary transportation requirements for rocket parts.
* The sites are close the coast line. This is evident with the many rocket landing tests on water bodies like the ocean.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


<a href="https://www.linkedin.com/in/molomunyansanga/">Molo Munyansanga</a> is a Data Science enthusiast with certificates in Statistics, Databases and Machine Learning. He is also enrolled in the Data Science Professional Certification program by IBM.

[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2022-03-15        | ....     | Molo. M        | Completed tasks |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
